## Preparar datos
# Leer Dataset

En el caso de este tutorial, nuestro Input seran los comentarios de los usuarios, mientras que el Output seran los sentimientos relacionados a las palabras en esos reviews

```
Input(x) -> Comentarios
Output(y) -> Sentimientos
```

In [1]:
import pandas as pd

In [4]:
# importar los datos desde el CSV

df_review = pd.read_csv('./dataset/IMDB Dataset.csv')
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
df_review.value_counts('sentiment')

sentiment
negative    25000
positive    25000
dtype: int64

In [16]:
#Estamos desbalanceando los datos para poder mas adelante entender tecnicas de balance de datos
df_positivo = df_review[df_review['sentiment'] == 'positive'][:9000]
df_negativo = df_review[df_review['sentiment'] == 'negative'][:1000]

# desbalanceado
df_review_des = pd.concat([df_positivo, df_negativo])
df_review_des.value_counts('sentiment')

sentiment
positive    9000
negative    1000
dtype: int64

## Balanceando el Dataset

In [17]:
from imblearn.under_sampling import RandomUnderSampler

#Lo que hace esta funcion es un re sample basado en la cantidad de datos en la clase menor. 
#En este caso se reducen los casos positivos hasta que quedan solo 1000
# https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html
rus = RandomUnderSampler()
df_review_bal, df_review_bal['sentiment'] = rus.fit_resample(df_review_des[['review']],
                                                             df_review_des['sentiment'])

df_review_bal.value_counts(['sentiment'])

sentiment
negative     1000
positive     1000
dtype: int64

## Separando data para testing y training

In [18]:
from sklearn.model_selection import train_test_split

#train test split recibe un dataset y lo divide entre datasets para entrenar y testear. Aqui estamos seleccionando solo 33% de los datos para testing
train, test = train_test_split(df_review_bal, test_size=0.33, random_state=42)

In [19]:
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

## Representacion del texto
- Count Vectorizer
- Tfidf

### Count Vectorizer

In [22]:
# Count Vectorizing significa vectorizar el conteo de datos, es decir exactamente cuantos datos existen en cada uno de los reviews
from sklearn.feature_extraction.text import CountVectorizer
text = ["Amo escribir código en Python. Amo el código en Python",
        "Odio escribir código en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text': text})
cv = CountVectorizer()
cv_matrix = cv.fit_transform(df['text'])
df_dtm = pd.DataFrame(
    cv_matrix.toarray(), index=df['review'].values, columns=cv.get_feature_names_out())
df_dtm

,amo,código,el,en,escribir,java,odio,python
review1,2,2,1,2,1,0,0,2
review2,0,2,1,2,1,2,2,0


### TFIDF

In [26]:
# El TFIDF indica la relevancia para la palabra.
# Una palabra representativa es aquella palabra que aparece multiples veces dentro de su propio review, pero pocas veces (o nada) en otros textos 
from sklearn.feature_extraction.text import TfidfVectorizer
df = pd.DataFrame({'review': ['review1', 'review2'], 'text': text})
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text'])
df_dtm = pd.DataFrame(tfidf_matrix.toarray(), index=df['review'].values, columns=tfidf.get_feature_names_out())
df_dtm

,amo,código,el,en,escribir,java,odio,python
review1,0.553373,0.393729,0.196865,0.393729,0.196865,0.000000,0.000000,0.553373
review2,0.000000,0.393729,0.196865,0.393729,0.196865,0.553373,0.553373,0.000000


### Transformacion de datos textuales a datos numericos

In [27]:
# Con esto, se crea un vector de datos que incluye el TFIDF de los datos que vamos a usar para entrenar el modelo
tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)

test_x_vector = tfidf.transform(test_x)
train_x_vector

<1340x19926 sparse matrix of type '<class 'numpy.float64'>'
	with 113038 stored elements in Compressed Sparse Row format>

## Seleccion del Modelo
#### Algoritmos de ML
1. Aprendizaje Supervisado (Supervised learning): Regresion (output numerico), Clasificacion (output discreto)
    - Input: Review
    - Output: Sentiment (discrete)
2. Aprendizaje no supervisado

### Support Vector Machine (SVM)

In [28]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y)

SVC(kernel='linear')

### Testeo

In [29]:
# Esto recibe una cantidad de inputs y predice el sentimiento que se le asocia a ese review
print(svc.predict(tfidf.transform(['A good movie'])))  # buena pelicula
print(svc.predict(tfidf.transform(['An excellent movie']))) # excelente pelicula
print(svc.predict(tfidf.transform(
    ['"I did not like this movie at all I gave this movie away"'])))  # no gusto

['positive']
['positive']
['negative']


### Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

DecisionTreeClassifier()

### Naive Bayes

In [31]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y)

GaussianNB()

### Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_x_vector, train_y)

LogisticRegression()

## Evaluacion del Modelo
### Score (Accuracy)

In [34]:
# El SCORE es calculado sumando los verdaderos positivos y negativos y dividiendolos sobre el total de datos. Nos da una idea de cuantos aciertos tuvo cada modelo

print(svc.score(test_x_vector, test_y))
print(dec_tree.score(test_x_vector, test_y))
print(gnb.score(test_x_vector.toarray(), test_y))
print(lr.score(test_x_vector, test_y))

0.8409090909090909
0.6590909090909091
0.6060606060606061
0.8363636363636363


### F1 Score
```
F1 Score = 2(Recall Precision) / (Recall + Precision)
```
- Precision = La cantidad de predicciones que resultan en verdaderos positivos sobre el total de predicciones relevantes
- Recall =  El porcentaje de resultados relevantes correctamente clasificados por el algoritmo

Calcularemos el F1 score del SVC Ya que fue el modelo mas exactitud

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/1200px-Precisionrecall.svg.png" alt="drawing" style="width:200px;"/>

In [36]:
from sklearn.metrics import f1_score

f1_score(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'],
         average=None)

array([0.84490399, 0.83670295])

### Reporte de clasificacion

In [37]:
from sklearn.metrics import classification_report

print(classification_report(test_y, svc.predict(test_x_vector),
                            labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.84      0.85      0.84       335
    negative       0.85      0.83      0.84       325

    accuracy                           0.84       660
   macro avg       0.84      0.84      0.84       660
weighted avg       0.84      0.84      0.84       660



### Matriz de Confusion

In [38]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, svc.predict(test_x_vector),
                 labels=['positive', 'negative'])

array([[286,  49],
       [ 56, 269]], dtype=int64)

Las columnas representan los valores Reales mientras que las filas representan los valores predichos
|__________| Positivo | Negativo |
|----------|----------|----------|
| Positivo | 286 | 49 |
| Negativo | 56 | 269 |